# "A Neural Probabilistic Language Model"
> Neural Network to model the language modeling task and to learn the distributed word representation with it
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [nnlm, torch, language model, nlp]


We will go through the paper by Yoshua Bengio - [A Neural Probabilistic Language Model]( http://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf)

## Language Modeling
In language modeling task we try to learn the joint probability function of word sequences p(w1 w2 .. wm). 

p(w1 w2 .. wm) = count(w1 w2 .. wm) / count(all m-gram word seq)

## Unseen Word sequences
Calculating joint probability is difficult becuase it involves counting all the m-word sequences and also, it doesn't generalize well on unseen word sequences. 

What will happen to the word sequences we have not seen during the modeling time? We will assign a zero probability to all such sequences.

We use probability chain formula to frame the joint probability as a product of conditional probabilities. We define joint probability as product of conditional probabilities as follows

p(w1 w2 ..wm) = p(w1) p(w2|w1) p(w3|w2w1) .. p(wm|w1..wm-1)

Calculating conditional probability is comparatively easy.   
p(w3|w2 w1) = count(w1 w2 w2)/count(w1 w2)
    
But still we have the same problem when calculating conditional probability of long word sequences - it is highly likely that we won't have seen such long sequence of exact words during the modeling time. How do we handle this?  

## N-gram assumption
Linguists have observed that a word depends only n previous words and not all the words before it. This is the n-gram or Markov assumption. This simplifies the function p(wm|w1 w2..wm-1) into p(wm| wm-n ..wm-1). If we make a trigram(n=3) assumption, which is popular in statistical language modeling, it becomes p(wm| wm-1 wm-2), i.e. each word is assumed to be dependent on only last two words in the sequence.
    
## Back-off and Smoothed Models
To further improve the generalization on unseen word sequences, there a few more bag of tricks like back-off trigram model where if p(w4|w3 w2) is not known to us, we consider futher smaller sequences until we find the required probability. We look at p(w4|w3) if it is available, else we look at p(w4). Also, we use smoothed trigram model to distribute the probability mass.
    

## Bengio's NNLM paper 
It addresses two problems with the traditional statistical language modelling 

1. Curse of Dimensionality  
To model a joint distribution of 10-gram word sequence of 10,000 words vocabulary, there are 10,000^10 -1 free parameters required. When modeling continous variables, we obtain generalization more easily because function to be learned is expected to have some local smoothness, i.e. if we have smoothness property for language modeling task, we should be able to use some local smoothness to generalize the model to unseen word sequences. 
If we have seen a word sequence "The cat is walking in the bedroom", it should be able to generalize to simliar word sequence like "A dog was running in the room" as "dog" and "cat" have similar semantic and grammatic roles.      The generalization in the statistical language modeling is obtained by gluing together(product of conditional probabilities) the short subsequneces. Typically, trigrams were considered becuase of the curse of dimensionality. It doesn't take in to account more than 1-2 previous words.
  
  
2. Not taking into account similarity between words  
The big problem in language modeling is generalization. If the model understands the semantic simliary between words, it will generalize better.
  
    
## Previous Work 
- <b>Neural Networks for Modeling Joint Probability Distribution-</b> Neural networks have been used to model the joint probability distribution of random variables and also for lanugage modeling before this paper. Each output node spits the conditional probability of a word give the sequence of words as input. 

- <b>Word Similarities</b> Discovering the word simliarties to obtain generalization has been tried as well, by clustering similar words together but the model proposed by Bengio learns the distributed feature vector to represent word similarty. 
    
- <b>Vector space representation of words -</b>The previous works in information retrieval has explored the vector-space representation of words(LSI) but this paper explores the reprsentation of words which helps in representing the probability distribution of the word sequences. The paper mentions that learning the word representation and probability distribution jointly is very useful.

![NNLM Model](nnlm.png)

## Implementation

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [19]:
corpus = ['i like dog', 'i love coffee', 'i hate milk']  # list of all the sentences
vocabulary = list(set(' '.join(corpus).split()))  # |v| list of all the uwords
word2int = {w:i for i, w in enumerate(vocabulary)}
corpus = ['i like dog', 'i love coffee', 'i hate milk']  # list of all the sentences
vocabulary = list(set(' '.join(corpus).split()))  # |v| list of all the uwords
n_vocab = len(vocabulary)

class NNLM(nn.Module):

    def __init__(self):
        super(NNLM, self).__init__()

        # projection layer
        self.C = nn.Embedding(n_vocab, m)

        # hidden layer
        # tanh(XH + d)
        self.H = nn.Parameter(torch.randn(n_step * m,
                              n_hidden).type(torch.FloatTensor))
        self.d = \
            nn.Parameter(torch.randn(n_hidden).type(torch.FloatTensor))

        # hidden layer
        # WX + b
        self.W = nn.Parameter(torch.randn(n_step * m,
                              n_vocab).type(torch.FloatTensor))
        self.b = \
            nn.Parameter(torch.randn(n_vocab).type(torch.FloatTensor))

        # tanh U
        self.U = nn.Parameter(torch.randn(n_hidden,
                              n_vocab).type(torch.FloatTensor))

    def forward(self, X):
        X = self.C(X)  # (batch_size, n_step, m)
        X = X.view(-1, n_step * m)  # (batch_size, n_step * m)
        tanh = torch.tanh(self.d + torch.mm(X, self.H))

        output = torch.mm(tanh, self.U) + torch.mm(X, self.W) + self.b
        return output


def prepare_input(sentences):
    X = []
    Y = []
    for sent in sentences:
        sent = sent.split()
        X.append([word2int[word] for word in sent[:-1]])
        Y.append(word2int[sent[-1]])
    return (X, Y)


In [20]:
X, Y = prepare_input(corpus)
m = 2
n_step = 2
n_hidden =2
model = NNLM()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
X = torch.LongTensor(X)
Y = torch.LongTensor(Y)

for epoch in range(5000):
    optimizer.zero_grad()
    output = model(X)
    # output : [batch_size, n_class], target_batch : [batch_size] (LongTensor, not one-hot)
    loss = criterion(output, Y)
    if (epoch + 1)%1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

Epoch: 1000 cost = 0.207585
Epoch: 2000 cost = 0.035829
Epoch: 3000 cost = 0.012441
Epoch: 4000 cost = 0.005522
Epoch: 5000 cost = 0.002739


In [21]:
input = []
for w in X:
    input.append([vocabulary[i] for i in w])
predictions = model(X).data.max(1, keepdim=True)[1].squeeze()

for i,o in zip(input, [vocabulary[pred] for pred in predictions]):
    print('Input: '+' '.join(i))
    print('Output:',o)
    print('\n')

Input: i like
Output: dog


Input: i love
Output: coffee


Input: i hate
Output: milk




## How word2vec improves on the NNLM model?

The embedding to hidden and hidden to softmax layers are expensive in NNLM network. The softmax layers has 300*|V| weights. For a vocabulary of 10,000 words, the number of weights will be approx 3 million. Word2vec simplifies the NNLM model in few ways.

- use negative sampling to update only a fraction of 3M weights in output layer.
- modeling a binary classification instead of next word prediction model